In [1]:
import numpy as np
from astropy.table import QTable
import astropy.coordinates as coord
import astropy.units as u

from galpy.potential import NFWPotential, HernquistPotential, MiyamotoNagaiPotential
from galpy.potential import evaluatePotentials as evalPot
from galpy.orbit import Orbit

# Importing data and transforming to Galactic Coordinates

In [2]:
# importing the data as an astropy table
# the data has been downloaded with the following criteria:
# - non null radial velocities
# - parallax_over_error > 5
# - ruwe <1 .4
# - parallax > 0.183 (to account for the parallax zero point of 0.017 mas)
gaia_rad = QTable.read("1629912465318O-result.csv")
#print(len(gaia_rad))

In [4]:
# Making a SkyCoord object to easily transform between coordinates
gaia_rad = gaia_rad[0:5]
c = coord.SkyCoord(ra = gaia_rad['ra']*u.deg,
                   dec = gaia_rad['dec']*u.deg,
                   distance = coord.Distance(parallax = (u.Quantity(gaia_rad['parallax'] ) +0.017)*u.mas),
                   pm_ra_cosdec = gaia_rad['pmra']*u.mas/u.year,
                   pm_dec = gaia_rad['pmdec']*u.mas/u.year,
                   radial_velocity = gaia_rad['dr2_radial_velocity']*u.km/u.s,
                   frame = 'icrs')


# Astropy computation

In [5]:
# transforming data to galactocentric coordinate system
galactocent = c.transform_to(coord.Galactocentric())

def Ang_Mom(coords):
    c = coords
    L_x =  (c.y * c.v_z - c.z * c.v_y)
    L_y =  (c.z * c.v_x - c.x * c.v_z)
    L_z = -(c.x * c.v_y - c.y * c.v_x) # sign is flipped by convention
    L_T = np.sqrt(L_x**2+L_y**2+L_z**2)
    return L_T, L_z # in pc * km/s 

def Energy(coords):
    c = coords
    G = 4.301 * 10**(-6) *u.kpc* (u.kilometer/u.s)**2/u.solMass
    M_h = 10**12 *u.solMass
    M_d = 9.3*10**10 *u.solMass
    M_b = 3*10**10 *u.solMass
    r_s = 21.5 *u.kpc 
    a_d = 6.5 *u.kpc  
    b_d = 0.26 *u.kpc 
    c_b = 0.7 *u.kpc
    
    HaloPot = NFWPotential(amp = G*M_h/(np.log(2)-0.5), a = r_s)
    DiskPot = MiyamotoNagaiPotential(amp = G*M_d, a = a_d, b = b_d)
    BulgePot = HernquistPotential(amp = 2*G*M_b, a = c_b)
    
    R = np.sqrt(c.x**2 + c.y**2)*u.kpc/(1000*u.pc)
    z = c.z *u.kpc/(1000*u.pc)
    Halo = evalPot(HaloPot, R, z) # in km^2/s^2
    Disk = evalPot(HaloPot, R, z) # in km^2/s^2
    Bulge = evalPot(HaloPot, R, z)# in km^2/s^2
    
    Potential = Halo + Disk + Bulge
    Kinetic = (c.v_x**2+ c.v_y**2 + c.v_z**2)/2
    
    return Kinetic , Potential # in km^2/s^2

In [6]:
a_EK , a_EP = Energy(galactocent) 
_, a_Lz = Ang_Mom(galactocent)
a_x = galactocent.x
a_y = galactocent.y
a_z = galactocent.z
a_vx = galactocent.v_x
a_vy = galactocent.v_y
a_vz = galactocent.v_z

# galpy computations

In [7]:
G = 4.301 * 10**(-6) *u.kpc* (u.kilometer/u.s)**2/u.solMass
M_h = 10**12 *u.solMass
M_d = 9.3*10**10 *u.solMass
M_b = 3*10**10 *u.solMass
r_s = 21.5 *u.kpc 
a_d = 6.5 *u.kpc  
b_d = 0.26 *u.kpc 
c_b = 0.7 *u.kpc
   
HaloPot = NFWPotential(amp = G*M_h/(np.log(2)-0.5), a = r_s)
DiskPot = MiyamotoNagaiPotential(amp = G*M_d, a = a_d, b = b_d)
BulgePot = HernquistPotential(amp = 2*G*M_b, a = c_b)
Potential = HaloPot + DiskPot + BulgePot

orbits =Orbit(c)# Orbit(c, ro = 8.2, vo = 220)

In [8]:
g_E = orbits.E(pot = Potential)
g_Lz = orbits.Lz(pot = Potential)
g_x = orbits.x()
g_y = orbits.y()
g_z = orbits.z()
g_vx = orbits.vx()
g_vy = orbits.vy()
g_vz = orbits.vz()

# Positions

In [9]:
print('x astropy:')
print(a_x)
print('x galpy:')
print(g_x)
print('y astropy:')
print(a_y)
print('y galpy:')
print(g_y)
print('z astropy:')
print(a_z)
print('z galpy:')
print(g_z)

x astropy:
[-7291.79873026 -6146.28502723 -7961.56083266 -7036.97018911
 -7360.42704925] pc
x galpy:
[7.16982355 6.02431885 7.83958711 6.91500075 7.23844744]
y astropy:
[646.85027776 314.36184366 124.97148677 171.9855855   33.26166494] pc
y galpy:
[0.64685028 0.31436184 0.12497149 0.17198559 0.03326166]
z astropy:
[  67.15137965 -163.38876594   30.02008493  -79.77193301  180.68448326] pc
z galpy:
[ 0.06711896 -0.16346591  0.03001382 -0.0798143   0.18065475]


# velocities

In [10]:
print('vx astropy:')
print(a_vx)
print('vx galpy:')
print(g_vx)
print('vy astropy:')
print(a_vy)
print('vy galpy:')
print(g_vy)
print('vz astropy:')
print(a_vz)
print('vz galpy:')
print(g_vz)

vx astropy:
[36.05557942 14.14958241  2.84078438  0.05634721 22.04829723] km / s
vx galpy:
[-34.25530231 -12.34926794  -1.04011258   1.74387036 -20.2477738 ]
vy astropy:
[244.61189455 202.05636312 231.72172406 263.7569165  209.18537627] km / s
vy galpy:
[231.25189455 188.69636312 218.36172406 250.3969165  195.82537627]
vz astropy:
[ 0.68364307 -0.27380154 -9.42565279  2.2075512  -5.62559234] km / s
vz galpy:
[ 0.15273895 -0.80385032 -9.95526001  1.67805269 -6.15594954]


# Energies

In [14]:
print('E astropy:')
print( a_EP+a_EK.value)
print('E galpy:')
print(g_E)

E astropy:
[-2643532.57219632 -2711845.73096656 -2616442.29641945 -2653088.39225906
 -2649921.74749142]
E galpy:
[-922832.01932987 -958137.81929638 -912887.66379116 -924874.99605356
 -929746.20576072]


# Angular Momenta

In [15]:
print('Lz astropy:')
print(a_Lz)
print('Lz galpy:')
print(g_Lz)

Lz astropy:
[1806983.26361328 1246344.08809448 1845221.61941412 1856059.24951098
 1540427.06488123] km pc / s
Lz galpy:
[1680.19333246 1140.64919672 1711.99574096 1731.19494503 1418.14516839]
